In [1]:
import requests
import datetime

In [14]:
"""This class converts a dict to nested objects"""

class Struct(object):
    """
    Attributes will depend on the structure of object.
    If we keep calling the 'everything' newsapi, then the attributes will be:
    
                    articles: A list of articles, each with their own objects
                    status: Status of request, should be 'ok'
                    totalResults: The total number of results available for the request, will need
                                  to use the &page= parameter to get these as only 20 articles are
                                  returned per request.
                                  
    Resource: https://stackoverflow.com/questions/1305532/convert-python-dict-to-object
    """
    def __init__(self, data):
        for name, value in data.items():
            setattr(self, name, self._wrap(value))

    def _wrap(self, value):
        if isinstance(value, (tuple, list, set, frozenset)): 
            return type(value)([self._wrap(v) for v in value])
        else:
            return Struct(value) if isinstance(value, dict) else value


"""
General class to aggregate all useful objects. Could customize, e.g. change structure of raw_data
to affect data pbject
"""
class myclass(object):
    """
    Attributes:
                call: The url sent to newsapi
                raw_data: The dictionary returned when requesting call
    
    Resource: https://stackoverflow.com/questions/1305532/convert-python-dict-to-object
    """
    
    def get_raw_data(self,call):
        r = requests.get(self.call).json()
        
        for i in r['articles']:
            del i['author']
            del i['urlToImage']
            t = datetime.datetime.strptime(i['publishedAt'], "%Y-%m-%dT%H:%M:%S%fZ")
            nt = t.replace(hour=0, minute=0, second=0, microsecond=0)
            i['publishedAt'] = str(nt)
            i['source'] = i['source']['name']
        
        return r
    
    def __init__(self,call):
        self.call = call
        self.data = Struct(self.get_raw_data(call))
        self.data.n_pages = int(self.data.totalResults/20)


In [15]:
n =  myclass('https://newsapi.org/v2/everything?q=(BTC OR bitcoin)&from=2018-01-11&to=2018-01-13&language=en&sortBy=popularity&page=20&apiKey=6d00cdefd3bc4ee38f8a7af69ac5bec4')

In [16]:
n.__dict__

{'call': 'https://newsapi.org/v2/everything?q=(BTC OR bitcoin)&from=2018-01-11&to=2018-01-13&language=en&sortBy=popularity&page=20&apiKey=6d00cdefd3bc4ee38f8a7af69ac5bec4',
 'data': <__main__.Struct at 0x7f9d2c061e48>}

In [17]:
n.data.__dict__

{'articles': [<__main__.Struct at 0x7f9d2c076128>,
 'n_pages': 88,
 'status': 'ok',
 'totalResults': 1778}

In [18]:
n.data.articles[0].__dict__


{'description': 'A Bitcoin conference has stopped taking the cryptocurrency as payment for tickets because of skyrocketing transaction fees and long processing time.',
 'publishedAt': '2018-01-11 00:00:00',
 'source': 'The Telegraph',
 'title': 'Major cryptocurrency conference no longer accepting Bitcoin payments',
 'url': 'http://www.telegraph.co.uk/technology/2018/01/11/major-cryptocurrency-conference-no-longer-accepting-bitcoin/'}